## Import Modules

In [12]:
import pandas as pd
import numpy as np

import albumentations as A
import cv2

import torch
from ultralytics import settings
from ultralytics import YOLO


# Module Config

In [4]:
# Ultralytics settings
print(settings) 

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")

{'settings_version': '0.0.4', 'datasets_dir': 'D:\\Tom\\GitHub\\Third Year Project\\datasets', 'weights_dir': 'D:\\Tom\\GitHub\\Third Year Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'D:\\Tom\\GitHub\\Third Year Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True


# Augmentation Methods

In [ ]:
def read_label(label_path):

    # Placeholder for reading the label file in YOLO format
    pass

def write_label(label_path, bboxes, image_shape):
    
    # Placeholder for writing the label file in YOLO format
    pass

def augment_image(image_path, label_path, augmentation):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bboxes = read_label(label_path)

    # Note that bboxes should be normalized (i.e., in the range [0, 1])
    transformed = augmentation(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    # Denormalize bboxes here if your write_label function expects that
    write_label(label_path, transformed_bboxes, transformed_image.shape)

    cv2.imwrite(image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))

# Zoom in/out
def zoom(image_path, label_path, min_zoom=0.8, max_zoom=1.2):
    augmentation = A.Compose([
        A.RandomScale(scale_limit=(min_zoom - 1, max_zoom - 1), p=1.0)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    augment_image(image_path, label_path, augmentation)

# Horizontal and Vertical Reflection
def reflection(image_path, label_path):
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    augment_image(image_path, label_path, augmentation)

# Rotation
def rotate(image_path, label_path, angle_limit=90):
    augmentation = A.Compose([
        A.Rotate(limit=angle_limit, p=1.0)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    augment_image(image_path, label_path, augmentation)

# Contrast and Brightness
def adjust_contrast_brightness(image_path, label_path, brightness_limit=0.2, contrast_limit=0.2):
    augmentation = A.Compose([
        A.RandomBrightnessContrast(brightness_limit=brightness_limit, contrast_limit=contrast_limit, p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# Histogram Equalization
def histogram_equalization(image_path, label_path):
    augmentation = A.Compose([
        A.HistogramMatching(p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# White Balance (using FancyPCA for color augmentation which can simulate white balance shifts)
def white_balance(image_path, label_path):
    augmentation = A.Compose([
        A.FancyPCA(alpha=0.1, p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# Sharpen
def sharpen(image_path, label_path):
    augmentation = A.Compose([
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# Gaussian Noise
def gaussian_noise(image_path, label_path, var_limit=(10.0, 50.0)):
    augmentation = A.Compose([
        A.GaussNoise(var_limit=var_limit, p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)


# Test Parameters

In [ ]:
# parameters for each specfic test

# Training

# Own Implementation of Object Detection System